In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
import IPython.display as Display
from IPython.display import display, display_markdown
from ipywidgets import Layout, Label
from pathlib import Path

import parmed as pmd
import re

from scipy.ndimage import gaussian_filter


np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   

# Width of elements in site parameter grid
grid_width = '3cm'
ggap = '0.5cm'
grid_total_width = '14cm'

forbidden_strings = ["..", "/", "\\", " ", "~"]


In [3]:
sys.path.append('./main/')
import phutil

# Make data storage directory
HOME = os.environ['HOME']
DATADIR = phutil.get_data_dir()

phutil.build_data_folders(DATADIR)

In [105]:


#################################################################
# Spectrum window setup
#################################################################


def format_coord(x, y):
    #return 'x=%.1f, y=%1.2f'%(x, y)
    return ''


def build_axes():
    specax.set_yticks([])
    specax.set_xlabel('$\\bar\\nu$ (cm$^{-1}$)')
    plt.tight_layout()
    plt.show()
        
vaxis = np.arange(-3000, 3000, dtype=float)

specout = widgets.Output(layout=Layout(width='4.25in',
                                      height='4.25in'))



#################################################################
# Spectrum calculation setup
#################################################################


# def calculate_spec():
    

calc_button = widgets.Button(
    description='Calculate',
    disabled=True,
    tooltip='Click to calculate spectrum',
)


def calc_button_onclick(b):
    if export_cb.value==True and export_prefix_is_valid()==False:
        msglbl.value += ' Please select a valid file name or deselect \'Save on finish?\''
        return
    msglbl.value = ''
    calculate_spec()
    
    if export_cb.value==True:
        export_button_onclick(0)
            
calc_button.on_click(calc_button_onclick)


homog_slider = widgets.IntSlider(
    value=10,
    min=5, # min value
    max=50, # max value
    step=1, # exponent step
    description='FWHM (1/cm)',
    disabled=False,
    readout_format='d',
    layout=Layout(width='11cm',
                 margin='5pt')
)
homog_slider.style.description_width='3cm'

homog_fwhm = int(round(homog_slider.value))
def homog_fwhm_on_change(v):
    global homog_fwhm
    homog_fwhm = int(round(v['new']))
homog_slider.observe(homog_fwhm_on_change, names='value')


specfilelbl = widgets.Label(value='Output file prefix: ')

specfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='5cm'),
    disabled=False
)


exportover_cb = widgets.Checkbox(
    value = False,
    disabled = False,
    description = 'Overwrite Existing?',
    layout = widgets.Layout(width='4cm')
)
exportover_cb.style.description_width='0pt'

export_button = widgets.Button(
    description='Save Now',
    layout=Layout(width='3cm'))

def export_prefix_is_valid():
    prefix = specfile.value
    fbase = DATADIR + "/spec/" + prefix
    
    Suffixes = ['_abs.txt', '_cd.txt']
    for suf in Suffixes:
        fname = fbase + suf
        for chars in forbidden_strings:
            if len(prefix)==0 or prefix.find(chars)!=-1:
                msglbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
                return False

        if os.path.isfile(fname)==True:
            if exportover_cb.value==False:
                msglbl.value = "Output file exists. Please check 'Overwrite Existing' or pick another export name."
                return False
            else:
                !{"rm " + fname}
            
    return fbase

def export_button_onclick(b):
    fbase = export_prefix_is_valid()
    if fbase!=False:
        np.savetxt(fbase + '_abs.txt', np.vstack([vaxis, abs_line.get_ydata()]).T)
        np.savetxt(fbase + '_cd.txt', np.vstack([vaxis, cd_line.get_ydata()]).T)
        msglbl.value = 'Spectrum exported under prefix ' + fbase
export_button.on_click(export_button_onclick)

homog_lbl = widgets.HTML(
    value = '<p style="font-size:12pt"><b>ZPL Width:</b></p>'
    
)

HomogBox = widgets.Box([homog_lbl, homog_slider], 
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width,
                                    border='solid 0.5pt gray',
                                    padding='0.1cm'))


########################################################################
# Components
########################################################################


# sdclass is either
# -- 'J' indicating J(w) form
# -- 'C' indicating C(w) form
# where C(w) = w*w*J(w)
#
# func(v, params) is a function that returns the (unnormalized)
# spectral density as a function of v (in cm-1). 
#
# pnames stores the names of each parameter upon which 
# func depends.
#
# pbounds defines the upper and lower bounds for each parameter
class specdens:
    def __init__(self, name, sdclass, func, pnames, pdefs, pbounds):
        self.name = name
        self.sdclass = sdclass
        self.func = func
        self.pnames = pnames
        self.pdefs = pdefs
        self.pbounds = pbounds
        
        
        self.label = widgets.HTML(
            value = '<p style="font-size:12pt"><b>' + name + '</b></p>',
            layout = Layout(width='5cm')
        )
        
        # This is the widget that displays components
        self.CompsBox = widgets.VBox([])
        
        # This list keeps track of added components
        self.List = []
        
        # This is the add button for this spectral density
        self.add_button = widgets.Button(
            description='Add Component'
        )
        
        def add_component(b):
            
            if self.sdclass=='C':
                desc = '\\bar\\lambda'
                maxv = 500.0
                defval = 10.0
            else:
                desc = 'S'
                maxv = 30.0
                defval = 0.1
                
            newSFloat = widgets.BoundedFloatText(
                value=defval,
                min=0,
                max=maxv,
                step=0.01,
                description=r'\(' + desc + '\):',
                disabled=False,
                layout=Layout(width='2.25cm')
            )
            newSFloat.style.description_width='0.5cm'
            
            # This is the function used to delete lognormal components. 
            # It should only be called by the 'X' button defined
            # for each component. 
            def del_on_click(b):
                for item in self.List:
                    if item.children[-1]==b:
                        self.List.remove(item)
                        self.CompsBox.children = tuple(self.List)
            
            newDelBt = widgets.Button(
                description='x',
                layout=Layout(width='1.0cm')
            )
            newDelBt.on_click(del_on_click)
            
            wList = [newSFloat]
            for p in range(0, len(self.pnames)):
                newParFloat = widgets.BoundedFloatText(
                    value=self.pdefs[p],
                    min=self.pbounds[p][0],
                    max=self.pbounds[p][1],
                    step=0.01,
                    disabled=False,
                    description=r'\(' + pnames[p] + '\):',
                    layout=Layout(width='3cm')
                )
                newParFloat.style.description_width='1.1cm'
                wList.append(newParFloat)
                
            fill_width = 13.5 - 3*(len(wList)-1) - 2.25 - 1 - 0.1 - 1
            filler = widgets.HTML('', layout=Layout(width=str(fill_width)+'cm'))
            
            wList.append(filler)
            
            wList.append(newDelBt)
        
            self.List.append(widgets.Box(wList, 
                                    layout=Layout(flex_flow='row',
#                                     justify_content='space-around',
                                    width='13.5cm')))

            self.CompsBox.children = tuple(self.List)

        
        self.add_button.on_click(add_component)

        # This is the box displayed in the main window. 
        self.Box = widgets.Box([widgets.HBox([self.label, self.add_button], 
                                    layout=Layout(flex_flow='row', justify_content='space-between')),
              self.CompsBox
             ],
             layout=Layout(flex_flow='column',
                        width=grid_total_width,
                        border='solid 0.5pt gray',
                        padding='0.1cm')
        )



##############################################################



spec_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + "/spec/", text='Download Spectra'),
        layout=Layout(
            margin='10pt'
        ))

spec_process = widgets.HTML(HTMLButtonPrompt.format(link='Calculator.ipynb', text='Process Spectra'),
        layout=Layout(
            margin='10pt'
        ))



msglbl = widgets.HTML(
    value='',
    layout=Layout(width=grid_total_width)
)

def lndens(v, params):
    vcut = params[0]
    sigma = params[1]
    return np.exp(-((np.log(v/vcut))**2)/(2.0*sigma*sigma))*(v>0.0)
    
def bodens(v, params):
    damp = params[0]
    return (v>0.0)*v/(v**2 + damp**2)
    
def ohmdens(v, params):
    vcut = params[0]
    alpha = params[1]
    copower = params[2]
    return (v>0.0)*(np.power(v,alpha)*np.exp(-np.power(v/vcut, copower)))

    
LogNormal = specdens('Log-Normal', 'J', lndens, 
                     ['\\bar\\nu_{cut}', '\sigma'], 
                     np.array([30.0, 0.6]),
                     np.array([[0.0, 200.0],
                               [0.1, 10.0]]))

BrownOsc = specdens('Brownian Oscillator', 'C', bodens, 
                     ['\\bar\\gamma'], 
                     np.array([30.0]),
                     np.array([[0.0, 200.0]]))

SupOhm = specdens('Super-Ohmic', 'J', ohmdens, 
                     ['\\bar\\nu_{cut}', '\\alpha - 2', '\\xi'],
                     np.array([30.0, 1, 1]),
                     np.array([[0.0, 200.0],
                               [0.25, 5.0],
                               [0.25, 5.0]]))

SubOhm = specdens('Sub-Ohmic', 'C', ohmdens, 
                     ['\\bar\\nu_{cut}', '\\alpha', '\\xi'],
                     np.array([30.0, 1, 1]),
                     np.array([[0.0, 200.0],
                               [0.25, 5.0],
                               [0.25, 5.0]]))

sdList = [LogNormal, 
          BrownOsc,
          SupOhm,
          SubOhm
         ]



display(widgets.HBox([
    widgets.Box([specout,
                widgets.HBox([specfilelbl, specfile, export_button]),
                widgets.HBox([exportover_cb]),
                widgets.HBox([spec_down, spec_process]),
                widgets.HBox([calc_button]),
                msglbl
                 ], layout=Layout(
                    flex_flow='column',
                    align_items='center',
                )),
        widgets.Box([
                    widgets.Label(layout=Layout(height='15px')),
                     HomogBox,
                     LogNormal.Box, 
                     BrownOsc.Box,
                     SupOhm.Box,
                     SubOhm.Box
                    ], layout=Layout(flex_flow='column', align_items='flex-start'))
]))

with specout:
    %matplotlib notebook
    specfig = plt.figure(figsize=(4,3))
    specax = plt.gca()
    specax2 = specax.twiny()
    abs_line, = specax.plot(vaxis, 0.0*vaxis)
    cd_line, = specax.plot(vaxis, 0.0*vaxis)
    build_axes()
    


In [106]:
def build_profile(NormFac, params, class_flag, func):
    Jspec = np.zeros(np.shape(vaxis))
    Cspec = np.zeros(np.shape(vaxis))
    
    # Frequency step size (in 1/cm)
    vstep = vaxis[1] - vaxis[0]
    
    # Indices where vaxis is not positive
    npndcs = np.where(vaxis<=0)
    
    # Dummy vaxis with zero set to 1, just to avoid divide-by-zero errors. 
    # Later we need to set all densities to zero at all npndcs
    vaxisX = vaxis.copy()
    vaxisX[npndcs] = 1.0
    
    # J-type spectral densities are parameterized by Huang-Rhys factors (S). 
    # These are normalized directly by the area under J(w). 
    if class_flag=='J':
        Jspec = func(vaxisX, params)
        
        # All spectral densities should be exactly zero for non-positive frequencies
        Jspec[npndcs] = 0.0
        Jspec *= NormFac/( vstep*np.sum(Jspec) )
        
        # C(v) = v*v*J(v)
        Cspec = vaxis*vaxis*Jspec
        
    # C-type spectral densities are parameterized by reorganization energies (lambda). 
    # These must be integrated 
    elif class_flag=='C':
        print('C-type spectral density')
        Cspec = func(vaxisX, params)
        
        # All spectral densities should be exactly zero for non-positive frequencies
        Cspec[npndcs] = 0.0
        
        # NormSpec is equal to Cspec / vaxis *except* where vaxis==0. There NormSpec = 0. 
        NormSpec = np.divide(Cspec, vaxis, out=np.zeros_like(vaxis), where=vaxis!=0)
        
        # NormFac is the reorganization energy, in units of 1/cm. 
        # The spectral density now gets the correct reorganization energy:
        Cspec *= NormFac / ( vstep*np.sum(NormSpec) )
        
        # Jspec is Cspec/(v^2) except where vaxis==0. There we set Cspec = 0. 
        Jspec = np.divide(Cspec, np.power(vaxis,2), out=np.zeros_like(vaxis), where=vaxis!=0)
        
    return Jspec, Cspec
        


# Step through spectral density classes
for sd in sdList:
    
    # Step through individual components
    for comp in sd.CompsBox.children:
        
        # Extract parameters. 
        
        # First widget stores value for either S or Lambda, depending on sdclass
        Slambda = comp.children[0].value
        
        Nparams = len(sd.pnames)
        pvals = np.zeros((Nparams,))
        # Next Nparams widgets store parameter values
        for n in range(0, Nparams):
            pvals[n] = comp.children[n+1].value
            
        Jv, Cv = build_profile(Slambda, pvals, sd.sdclass, sd.func)
        
        plt.plot(vaxis, Jv)
        